In [1188]:
# import sympy
# import numpy as np
# import scipy.optimize
# import matplotlib.pyplot as plt
# import matplotlib.cm as cm
# import matplotlib.colors as colors

In [1189]:
import numpy as np
import sympy
import sympy as sy
from sympy import symbols, factorial, Function, summation, binomial, product
import scipy.optimize
import matplotlib.pyplot as plt
n, i, j, m = symbols('n, i, j, m', integer=True, real=True)

t = symbols('t')
P = Function('P') #Position Function as a function of time

deg = 6 #Bezier degree

B = summation(binomial(n, i)*(1 - t)**(n-i)*t**i*P(i), (i, 0, n))
P_vect = (sympy.Matrix([P(i) for i in range(deg)])).T #Bezier 6 uses range(6)?
P_vect

Matrix([[P(0), P(1), P(2), P(3), P(4), P(5)]])

In [1190]:
B #Bezier Summation

Sum(t**i*(1 - t)**(-i + n)*P(i)*binomial(n, i), (i, 0, n))

In [1191]:
Bez6 = B.subs(n,6).doit() #bezier n = 6
Bez6

t**6*P(6) + 6*t**5*(1 - t)*P(5) + 15*t**4*(1 - t)**2*P(4) + 20*t**3*(1 - t)**3*P(3) + 15*t**2*(1 - t)**4*P(2) + 6*t*(1 - t)**5*P(1) + (1 - t)**6*P(0)

In [1192]:
Bez6_d = sympy.diff(Bez6,t) #derivative of bezier n =6
# Bez6_d.collect(P_d[0]) #fix

In [1193]:
Bez6_d2 = sympy.diff(Bez6_d,t) #double deriv of bezier n=6
# Bez6_d2

In [1194]:
Bez6_d3 = sympy.diff(Bez6_d2,t) #double deriv of bezier n=6
# Bez6_d3

In [1195]:
P_vect

Matrix([[P(0), P(1), P(2), P(3), P(4), P(5)]])

In [1196]:
def difference_operator(P_vect, j=1): #check if j must be 1
    return sympy.Matrix([P_vect[i+1] - P_vect[i] for i in range(len(P_vect) - j)]).T
P_d = difference_operator(P_vect) #vel
P_d

Matrix([[-P(0) + P(1), -P(1) + P(2), -P(2) + P(3), -P(3) + P(4), -P(4) + P(5)]])

In [1197]:
## Compute P terms for each derivatives in Bezier domain
def P_deriv(deriv, n=3):
    P_vect = (sympy.Matrix([P(i) for i in range (6)])).T #i in range is Arbitary depending on Bezier_deriv
    P_deriv = P_vect #temporary derivative term
    for i in range(deriv):
        P_deriv = difference_operator(P_deriv)
    return P_deriv

deriv = 3 #for jerk
# P_deriv(deriv)

In [1198]:
class Bezier:
#https://en.wikipedia.org/wiki/B%C3%A9zier_curve

    def __init__(self, P, T: float):
        self.P = P
        self.m = P.shape[0]
        self.n = P.shape[1]-1
        self.T = T
    
    def eval(self, t):
        #https://en.wikipedia.org/wiki/De_Casteljau%27s_algorithm
        beta = t/self.T
        A = sympy.Matrix(self.P)
        for j in range(1, self.n + 1):
            for k in range(self.n + 1 - j):
                A[:, k] = A[:, k] * (1 - beta) + A[:, k + 1] * beta
        return A[:, 0]

    def deriv(self, m=1):
        D_sy = sympy.Matrix(self.P)
        diff_calc = sympy.zeros(D_sy.shape[0]-1,1).T
        for j in range(1, m+1): #should range start from 1 or 0???
            diff_calc = diff_calc.row_join(difference_operator(D_sy,j))
            D_soln = (self.n +1 -j)*diff_calc  #constant term is 5? or 4? is it P_vect.shape[1]-1?
        return Bezier(D_soln/self.T**m, self.T)
    


Bezier Path Planning in sympy

In [1199]:
import casadi as ca

class Bezier_ca:
#https://en.wikipedia.org/wiki/B%C3%A9zier_curve

    def __init__(self, P: ca.SX, T: float):
        self.P = P
        self.m = P.shape[0]
        self.n = P.shape[1]-1
        self.T = T
    
    def eval(self, t):
        #https://en.wikipedia.org/wiki/De_Casteljau%27s_algorithm
        beta = t/self.T
        A = ca.SX(self.P)
        for j in range(1, self.n + 1):
            for k in range(self.n + 1 - j):
                A[:, k] = A[:, k] * (1 - beta) + A[:, k + 1] * beta
        return A[:, 0]
    
    def deriv(self, m=1):
        D = ca.SX(self.P)
        for j in range(0, m):
            D = (self.n - j)*ca.horzcat(*[ D[:, i+1] - D[:, i] for i in range(self.n - j) ])
        return Bezier_ca(D/self.T**m, self.T)

In [1200]:
P_ca = ca.SX.sym('P', 1, 6)
B_ca = Bezier_ca(P_ca, 1)#.eval(0)

In [1201]:
display(P_ca)
display(P_vect)

SX([[P_0, P_1, P_2, P_3, P_4, P_5]])

Matrix([[P(0), P(1), P(2), P(3), P(4), P(5)]])

In [1202]:
# B_sy = Bezier(P_vect,1)
# B_sy_d = B_sy.deriv()
# B_sy_d.eval(0)

In [1203]:
def find_Q(deriv, poly_deg, n_legs):
    """
    Finds the cost matrix Q
    @param deriv: for cost J, 0=position, 1=velocity, etc.
    @param poly_deg: degree of polynomial
    @n_legs: number of legs in trajectory (num. waypoints - 1)
    @return Q matrix for cost J = p^T Q p
    """
    k, l, m, n, n_c, n_l = sympy.symbols('k, l, m, n, n_c, n_l', integer=True)
    # k summation dummy variable
    # n deg of polynomial

    beta = sympy.symbols('beta')  # scaled time on leg, 0-1
    c = sympy.MatrixSymbol('c', n_c, 1)  # coefficient matrices, length is n+1, must be variable (n_c)
    T = sympy.symbols('T')  # time of leg
    P = sympy.summation(c[k, 0]*sympy.factorial(k)/sympy.factorial(k-m)*beta**(k-m)/T**m, (k, m, n))  # polynomial derivative
    P = P.subs({m: deriv, n: poly_deg}).doit() #B
    J = sympy.integrate(P**2, (beta, 0, 1)).doit()  # cost
    p = sympy.Matrix([c[i, 0] for i in range(poly_deg+1)])  # vector of terms
    Q = sympy.Matrix([J]).jacobian(p).jacobian(p)/2  # find Q using second derivative
    assert (p.T@Q@p)[0, 0].expand() == J  # assert hessian matches cost
    
    Ti = sympy.MatrixSymbol('T', n_l, 1)
    return sympy.diag(*[
        Q.subs(T, Ti[i]) for i in range(n_legs) ])
    # return Q


#penalize jerk --> deriv = 3 for find_Q

In [1204]:
# find_Q(3,4,1)

Bezier Cost Function

$ p * Q * p.T$

p = coefficient matrix for each constraints

Q = Hessian Matrix derived in Bezier form penalizing jerk
$Q = intergral(Jerk)^2.jacobian(p).jacobian(p)/2$

In [1205]:
def find_Q_Bez(bez_deg, n_legs=1): #Bezier 6 = bez_deg =6
    """
    Finds the cost matrix Q
    @param deriv: for cost J, 0=position, 1=velocity, etc.
    @param poly_deg: degree of polynomial
    @n_legs: number of legs in trajectory (num. waypoints - 1)
    @return Q matrix for cost J = p^T Q p
    """
    k, l, m, n, n_c, n_l = sympy.symbols('k, l, m, n, n_c, n_l', integer=True)
    # k summation dummy variable
    # n deg of polynomial

    beta = sympy.symbols('beta')  # scaled time on leg, 0-1
    c = sympy.MatrixSymbol('c', n_c, 1)  # coefficient matrices, length is n+1, must be variable (n_c)
    T = sympy.symbols('T')  # time of leg
    t = sympy.symbols('t')  # evaluate at time t 
    P= Function('P')
    P_c = (sympy.Matrix([P(i) for i in range(bez_deg)])).T #Bezier Positon Matrices length = n     #must be variable (n_c)
    Bez6 = Bezier(P_c,T)
    B_d3 = Bez6.deriv().deriv().deriv()    # 3rd derivative for jerk; 4th deriv for snap
    
    Jerk = B_d3.eval(t)                         #evaluate Bezier at t
    J = (Jerk@Jerk.T).doit()                    #square of jerk term
    J = sympy.integrate(J, (t, 0, 1)).doit()    # cost
    
    p = sympy.Matrix([c[i, 0] for i in range(bez_deg+1)])  # vector of terms
    Q = sympy.Matrix([J]).jacobian(P_c).jacobian(P_c)/2  # find Q using second derivative, P_c or p for jacobian
    # assert (p.T@Q@p)[0, 0].expand() == J  # assert hessian matches cost

    Ti = sympy.MatrixSymbol('T', n_l, 1)
    
    return J, Q
        #     sympy.diag(*[
        # Q.subs(T, Ti[i]) for i in range(n_legs) ])
find_Q_Bez(6)[1]

Matrix([
[      3600/T**6 - 7200/T**7 + 7200/T**8 - 3600/T**9 + 720/T**10,     -10800/T**6 + 25200/T**7 - 28800/T**8 + 16200/T**9 - 3600/T**10,      10800/T**6 - 32400/T**7 + 44400/T**8 - 28800/T**9 + 7200/T**10,     -3600/T**6 + 18000/T**7 - 32400/T**8 + 25200/T**9 - 7200/T**10,   -3600/T**7 + 10800/T**8 - 10800/T**9 + 3600/T**10,   -1200/T**8 + 1800/T**9 - 720/T**10],
[-10800/T**6 + 25200/T**7 - 28800/T**8 + 16200/T**9 - 3600/T**10,    32400/T**6 - 86400/T**7 + 112800/T**8 - 72000/T**9 + 18000/T**10, -32400/T**6 + 108000/T**7 - 169200/T**8 + 126000/T**9 - 36000/T**10,  10800/T**6 - 57600/T**7 + 118800/T**8 - 108000/T**9 + 36000/T**10,  10800/T**7 - 37200/T**8 + 45000/T**9 - 18000/T**10,   3600/T**8 - 7200/T**9 + 3600/T**10],
[ 10800/T**6 - 32400/T**7 + 44400/T**8 - 28800/T**9 + 7200/T**10, -32400/T**6 + 108000/T**7 - 169200/T**8 + 126000/T**9 - 36000/T**10,  32400/T**6 - 129600/T**7 + 244800/T**8 - 216000/T**9 + 72000/T**10, -10800/T**6 + 64800/T**7 - 163200/T**8 + 180000/T**9 - 7200

Deriving Coefficient Matrix p from Bezier Constraints

In [1206]:
def derive_bezier6():
    n = 6 #Bezier Degree
    T = sympy.Symbol('T') #ca.SX.sym('T')
    t = sympy.Symbol('t') #ca.SX.sym('t')                       
    P= Function('P')       # P = ca.SX.sym('P', 1, n)
    P_c = (sympy.Matrix([P(i) for i in range(n)])).T #Bezier Positon Matrices length = n
    B = Bezier(P_c, T)

    # derivatives
    B_d = B.deriv()
    B_d2 = B_d.deriv()
    B_d3 = B_d2.deriv()
    B_d4 = B_d3.deriv()

    # boundary conditions

    # trajectory
    p = B.eval(t)
    v = B_d.eval(t)
    a = B_d2.eval(t)
    r = sy.Matrix.vstack(p,v,a)    #ca.vertcat(p, v, a)

    # given position/velocity boundary conditions, solve for bezier points
    wp_0 = sympy.MatrixSymbol('ptin', 2, 1) #ca.SX.sym('p0', 2, 1)  # pos/vel at waypoint 0
    wp_1 = sympy.MatrixSymbol('ptone', 2, 1) #ca.SX.sym('p1', 2, 1)  # pos/vel at waypoint 1

    constraints = []
    constraints += [(B.eval(0), wp_0[0])]  # pos @ wp0
    constraints += [(B_d.eval(0), wp_0[1])]  # vel @ wp0
    constraints += [(B_d2.eval(0), 0)]  # zero accel @ wp0
    constraints += [(B.eval(T), wp_1[0])]  # pos @ wp1
    constraints += [(B_d.eval(T), wp_1[1])]  # vel @ wp1
    constraints += [(B_d2.eval(T), 0)]  # zero accel @ wp1

    assert len(constraints) == 6

    Y = sy.Matrix.vstack(*[c[0] for c in constraints])  #ca.vertcat(*[c[0] for c in constraints])
    b = sy.Matrix([c[1] for c in constraints])   #ca.vertcat(*[c[1] for c in constraints])
    A = Y.jacobian(P_c)
    A_inv = A.inv()
    P_sol = (A_inv@b).T
    return {
        'bezier6_solve': P_sol,#.subs(wp_0,[1,2]).subs(wp_1,[1,2]) #ca.Function('bezier6_solve', [wp_0, wp_1, T], [P_sol], ['wp_0', 'wp_1', 'T'], ['P']),
        'bezier6_traj': r #ca.Function('bezier6_traj', [t, T, P], [r], ['t', 'T', 'P'], ['r']),
    }
    # return sympy.diag(*[
    #     Q.subs(T, Ti[i]) for i in range(n_legs) ])

In [1207]:
# #RUNNING BEZIER constraint coefficient matrix

# wp_0 = sympy.MatrixSymbol('ptin', 2, 1) #ca.SX.sym('p0', 2, 1)  # pos/vel at waypoint 0
# wp_1 = sympy.MatrixSymbol('ptone',2,1)

# #INPUTS
# wpVal0 = [0,1] #Waypoint value in [pos, vel] for wp0
# wpVal1 = [2,3] #Waypoint value in [pos, vel] for wp1
# # T_l = 4 #input Time T

# p_constr = derive_bezier6()['bezier6_solve'] #Bezier Coefficient
# for i in range(wp_0.shape[0]):
#     p_constr = p_constr.subs(wp_0[i],wpVal0[i]).subs(wp_1[i],wpVal1[i])
    
# # p_constr = p_constr.subs(T,T_l) #Substitute T value

# p_constr

In [1208]:
def find_cost_function(bez_deg, wpVal0, wpVal1, n_legs = 1):
    ''' 
    Takes bezier degree, initial waypoint, and final waypoint
    waypoints are [position 1, velocity 1] in sympy Matrix type
    outputs cost function, J_total
    '''
    T = sympy.Symbol('T')
    wp_0 = sympy.MatrixSymbol('ptin', 2, 1) #pos/vel at waypoint 0
    wp_1 = sympy.MatrixSymbol('ptone',2,1)    

    Q_Bez = find_Q_Bez(6,n_legs)[1]
    p_constr = derive_bezier6()['bezier6_solve'] #Bezier Coefficient
    for i in range(len(wpVal0)):
        p_constr = p_constr.subs(wp_0[i],wpVal0[i]).subs(wp_1[i],wpVal1[i])
    pBez =p_constr#.subs(T,T)

    # display(sympy.Matrix(Ti))
    #T could be fixed to vary with n_legs
    J_total = (((pBez@Q_Bez@pBez.T)[0,0]).simplify()) #Jtotal term for optimized cost calculation

    return J_total
find_cost_function(6,[0,1],[2,3],1)

48*(243*T**6 - 2106*T**5 + 8104*T**4 - 16260*T**3 + 17580*T**2 - 9720*T + 2160)/T**10

In [1209]:
# def find_opt_time(bez_deg, wpVal0, wpVal1, n_legs =1):
def find_opt_time(bez_deg, bc, n_legs = 1):
    '''
    Takes bezier degree, initial waypoint, and final waypoint
    waypoints are [[position], [velocity]]
    
    Using Scipy.Optimize function, outputs optimized leg time
    '''

    T = sympy.Symbol('T')
    Ti = sympy.symbols('T_0:{:d}'.format(n_legs))
    k = sympy.symbols('k') #weight on time
    t_temp = 1 ## Set temporary time eval, t for optimization calculate

    J_total_x = find_cost_function(bez_deg, bc[:,0, 0], bc[:, 1, 0], n_legs)
    J_total_y = find_cost_function(bez_deg, bc[:,0, 1], bc[:, 1, 1], n_legs)
    J_total_z = find_cost_function(bez_deg, bc[:,0, 2], bc[:, 1, 2], n_legs)

    J_total = J_total_x + J_total_y + J_total_z + k * sum(Ti)# Sum of all components' jerk term
    # n_legs =1
    J_total = J_total.subs(T,sy.Matrix(Ti)).doit()
    f_J = sympy.lambdify([Ti,k],J_total) ##RECHECK LAMBDIFY FUNCTION

    k_time = 10 #weight on time
    sol = scipy.optimize.minimize(lambda T: f_J(T,k_time), [1]*n_legs, bounds=[[0.1, 100]]* n_legs)
    T_opt = sol['x']
    
    return T_opt ## Optimzied T leg time for eval in Bezier
find_opt_time

<function __main__.find_opt_time(bez_deg, bc, n_legs=1)>

In [1210]:
bc = np.array([
    [ # position
        [1, 2, 3],  # wp0, x, y, z
        [2, 3, 4]   # wp1, x, y, z
    ],
    [ # velocity
        [1, 0, 1],
        [0, 1, -1]
    ]
])
find_opt_time(6,bc,1)


T_0*k + Matrix([[48*(540 + Matrix([[3*T_0**6 - 36*T_0**5 + 199*T_0**4 - 630*T_0**3 + 1290*T_0**2 - 1350*T_0]]))/T_0**10 + 48*(540 + Matrix([[12*T_0**6 - 144*T_0**5 + 721*T_0**4 - 1875*T_0**3 + 2640*T_0**2 - 1890*T_0]]))/T_0**10 + 48*(540 + Matrix([[27*T_0**6 - 234*T_0**5 + 931*T_0**4 - 2100*T_0**3 + 2730*T_0**2 - 1890*T_0]]))/T_0**10]])

array([2.31512528])

Finding the path of Multirotor in 3D space

In [1211]:
def multiRotor_plan(): ## Currently outputs optimized time
    bezier_6 = derive_bezier6()
    bc = np.array([
        [ # position
            [1, 2, 3],  # wp0, x, y, z
            [2, 3, 4]   # wp1, x, y, z
        ],
        [ # velocity
            [1, 0, 1],
            [0, 1, -1]
        ]
    ])

    time_opt = find_opt_time(6, bc)

    # t0 = np.linspace(0, time_opt, 1000)

    # ##Convert matrix Symbol and substiute aray
    # T = sympy.Symbol('T')

    # wp_0 = sympy.MatrixSymbol('ptin', 2, 1) #pos/vel at waypoint 0
    # wp_1 = sympy.MatrixSymbol('ptone',2,1)    

    # PX = derive_bezier6()['bezier6_solve'] #Bezier Coefficient
    # for i in range(len(bc[:,0,0])):
    #     PX = PX.subs(wp_0[i],bc[:,0,0][i]).subs(wp_1[i],bc[:,1,0][i])
    # PX = PX.subs(T,T)

    
    # traj_x = (derive_bezier6()['bezier6_traj'].subs(T,time_opt).subs(t,t0)).T
    return time_opt

t_opt = multiRotor_plan()

T_0*k + Matrix([[48*(540 + Matrix([[3*T_0**6 - 36*T_0**5 + 199*T_0**4 - 630*T_0**3 + 1290*T_0**2 - 1350*T_0]]))/T_0**10 + 48*(540 + Matrix([[12*T_0**6 - 144*T_0**5 + 721*T_0**4 - 1875*T_0**3 + 2640*T_0**2 - 1890*T_0]]))/T_0**10 + 48*(540 + Matrix([[27*T_0**6 - 234*T_0**5 + 931*T_0**4 - 2100*T_0**3 + 2730*T_0**2 - 1890*T_0]]))/T_0**10]])

In [1212]:
t_opt

array([2.31512528])

In [1213]:
np.average(t_opt)

2.3151252777563114